# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [155]:
# Details Student 1:
print(' Bar Sade\n','315340463')
# Details Student 2:


 Bar Sade
 315340463


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [175]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [176]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [177]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [178]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [179]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [180]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [181]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [182]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [183]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

#### This code performs several important preprocessing steps for text data and prepares the data for text classification using machine learning models.  
I used a function to remove unwanted chars which is applied to the 'story' column of both DataFrames using the .apply() method.  
As a result, all text in the 'story' column of both DataFrames undergoes the text cleaning operations and feature extraction using TF-IDF

In [188]:
def Remove_chars(text):
    text = re.sub(r'\d+', '', text)  # Removes numbers
    text = re.sub(r'[^\w\s]', '', text)  # Removes punctuation marks
    text = re.sub(r'\s+', ' ', text)  # Removes multiple blank spaces
    text = text.strip()  # Remove trailing spaces
    return text

# Preprocess text data using Remove_Chars function
df_train['story'] = df_train['story'].apply(Remove_chars)
df_test['story'] = df_test['story'].apply(Remove_chars)

# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_text = df_train['story']  # Separate text data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)

y_train = df_train['gender']

#### This code focuses on feature scaling, data splitting, and training machine learning models  
The first step is to perform feature scaling on the TF-IDF features extracted from the text data.  
Feature scaling is important to ensure that all features have the same scale, which can be crucial for certain machine learning algorithms.  
The data splitting - after feature scaling, the code splits the data into training and validation sets using the train_test_split function from Scikit-learn.
At last, in this code, I use Training Logistic Regression and Naive Bayes Models.  
At this point, I have successfully preprocessed the text data, scaled the features, and split the data into training and validation sets.  
The Logistic Regression and Multinomial Naive Bayes models have also been trained on the training data. These models can now be evaluated and used for making predictions on new data.

In [185]:
# Create MinMaxScaler for feature scaling
scaler = MinMaxScaler()

# Fit and transform the TF-IDF features with MinMaxScaler
X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())  # Convert to array for scaling

X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Train a Logistic Regression model on the training set
logistic_model = linear_model.LogisticRegression()
logistic_model.fit(X_train, y_train)

# Train a Multinomial Naive Bayes model on the training set
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train, y_train)

LogisticRegression()

MultinomialNB()

#### This section of the code is focused on testing the machine learning models (Logistic Regression and Naive Bayes) on a separate test dataset and displaying the predictions.
I have prepared the test data, used feature scaling for test data and predicted both models.  
After executing these lines of code, df_test will contain the original text data, TF-IDF features, as well as the predicted gender labels from both the Logistic Regression and Naive Bayes models.

In [186]:
X_test_text = df_test['story']  # Separate text data
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

# Transform the test data with MinMaxScaler
X_test_scaled = scaler.transform(X_test_tfidf.toarray())  # Convert to array for scaling

# Predict gender labels for test data using both models
df_test['predicted_gender_logistic'] = logistic_model.predict(X_test_scaled)
df_test['predicted_gender_naive_bayes'] = naive_bayes_model.predict(X_test_scaled)

#### This code displays predictions and Cross-Validation and Model Evaluation
Using cross validation technique which is used to evaluate the performance of machine learning models on different random splits of the training data.  
LR_score and NB_score performs cross-validation for each model.  
In addition, this code uses model evaluation on validation set to show the F1 scores for male and female and the average between them.  
I used a funtion to display predictions that displays predictions made by both models on the test data.  
This function prints the top 5, last 5, and the total counts of predicted gender labels for each model.  

In summary, this code block evaluates the performance of the Logistic Regression and Multinomial Naive Bayes models using cross-validation and provides an assessment of their accuracy and F1 scores on a validation set.  
It also displays the predictions made by both models on the test data.

In [187]:
# Using RepeatedKFold for evaluating the performance of our model by performing cross-validation on different random splits of data 
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
LR_score = cross_val_score(logistic_model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
NB_score = cross_val_score(naive_bayes_model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

# Evaluate models on the validation set
y_pred_logistic_val = logistic_model.predict(X_val)
y_pred_naive_bayes_val = naive_bayes_model.predict(X_val)

f1_logistic_val = f1_score(y_val, y_pred_logistic_val, average='weighted')
f1_naive_bayes_val = f1_score(y_val, y_pred_naive_bayes_val, average='weighted')

print("--- Logistic Regression ---")
print("Accuracy Score: %.4f" % np.mean(LR_score))
print("F1_male score: %.4f" % f1_male_logistic_val)
print("F1_female score: %.4f" % f1_female_logistic_val)
print("F1_Average Score: %.4f" % average_f1_logistic_val)

print("\n--- Naive Bayes ---")
print("Accuracy Score: %.4f" % np.mean(NB_score))
print("F1_male score: %.4f" % f1_male_naive_bayes_val)
print("F1_female score: %.4f" % f1_female_naive_bayes_val)
print("F1_Average Score: %.4f" % average_f1_naive_bayes_val)
print('\n')

def Display_Predictions(title, df_test, y_pred):
    # Create a DataFrame with the 'story' and 'predicted gender' columns
    df_predicted = pd.DataFrame({'story': df_test['story'], 'predicted gender': y_pred})
    
    print('[--- %s Predictions ---]' % title)
    print('<Top-5>')
    # Print the top 5 rows of the DataFrame
    print(df_predicted.head(5))
    print('')

    print('<Last-5>')
    # Print the last 5 rows of the DataFrame
    print(df_predicted.tail(5))
    print('')

    print('<Total>')
    # Print the counts of predicted gender labels
    print(df_predicted['predicted gender'].value_counts())
    print('')

# Display predictions for both models
Display_Predictions('Logistic Regression', df_test, LR_test_y_pred)
Display_Predictions('Naive Bayes', df_test, NB_test_y_pred)

--- Logistic Regression ---
Accuracy Score: 0.7736
F1_male score: 0.8689
F1_female score: 0.4483
F1_Average Score: 0.6586

--- Naive Bayes ---
Accuracy Score: 0.7614
F1_male score: 0.8889
F1_female score: 0.5424
F1_Average Score: 0.7156


[--- Logistic Regression Predictions ---]
<Top-5>
                                               story predicted gender
0  כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...                m
1  הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת י...                m
2  אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...                m
3  רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי אני ...                f
4  אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...                m

<Last-5>
                                                 story predicted gender
318  בשנה האחרונה הרגשתי די תקוע בעבודה השגרה הפכה ...                m
319  אני ואילן חברים טובים מזה שנה תמיד חלמנו לפתח ...                f
320  מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...           

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)